In [2]:
import os
from collections import defaultdict
from neo4j import GraphDatabase
import pandas as pd
import dotenv
pd.set_option('display.max_columns', None)

dotenv.load_dotenv()

# Configuration
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
CSV_PATH = "../data/processed/ViKG-NLQ-2-Cypher-data-cleaned.csv"
OUTPUT_PATH = "../data/processed/ViKG-NLQ-2-Cypher-data-validated.csv"

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI)
driver.verify_connectivity()
print(f"Connected to Neo4j at {NEO4J_URI}")

Connected to Neo4j at bolt://localhost:7687


In [3]:
def execute_query(query: str) -> dict:
    """Execute a Cypher query and return results info."""
    try:
        with driver.session() as session:
            result = session.run(query)
            records = list(result)
            return {"success": True, "result_count": len(records), "error": None}
    except Exception as e:
        return {"success": False, "result_count": 0, "error": str(e)}

# Load and validate queries
df = pd.read_csv(CSV_PATH)
print(f"Loaded {len(df)} queries from CSV")

# Validate each query
stats = {"total": 0, "successful": 0, "failed": 0, "with_results": 0, "empty_results": 0}
result_counts = []

for idx, row in df.iterrows():
    stats["total"] += 1
    query = row.get("answer", "")
    
    if not query or pd.isna(query):
        stats["failed"] += 1
        result_counts.append(-1)
        continue
    
    result = execute_query(query)
    
    if result["success"]:
        stats["successful"] += 1
        count = result["result_count"]
        result_counts.append(count)
        if count > 0:
            stats["with_results"] += 1
        else:
            stats["empty_results"] += 1
    else:
        stats["failed"] += 1
        result_counts.append(-1)
    
    if stats["total"] % 1000 == 0:
        print(f"Processed {stats['total']} queries...")

df["result_count"] = result_counts
print("Done!")

Loaded 5412 queries from CSV
Processed 1000 queries...
Processed 2000 queries...
Processed 3000 queries...
Processed 4000 queries...
Processed 5000 queries...
Done!


In [4]:
# Print statistics report
print("=" * 50)
print("QUERY VALIDATION REPORT")
print("=" * 50)

print(f"\nOVERVIEW")
print(f"  Total queries:        {stats['total']}")
print(f"  Successful execution: {stats['successful']} ({100*stats['successful']/stats['total']:.1f}%)")
print(f"  Failed execution:     {stats['failed']} ({100*stats['failed']/stats['total']:.1f}%)")

print(f"\nRESULT STATISTICS")
print(f"  With results:    {stats['with_results']} ({100*stats['with_results']/max(stats['successful'],1):.1f}%)")
print(f"  Empty results:   {stats['empty_results']} ({100*stats['empty_results']/max(stats['successful'],1):.1f}%)")

# Result count distribution
print(f"\nRESULT COUNT DISTRIBUTION")
successful_df = df[df["result_count"] >= 0]
bins = [0, 1, 2, 6, 11, 51, float('inf')]
labels = ["0", "1", "2-5", "6-10", "11-50", "50+"]
for i in range(len(bins)-1):
    count = len(successful_df[(successful_df["result_count"] >= bins[i]) & (successful_df["result_count"] < bins[i+1])])
    bar = "#" * (count * 40 // max(stats['successful'], 1))
    print(f"  {labels[i]:>5} results: {count:>5} {bar}")

print("=" * 50)

QUERY VALIDATION REPORT

OVERVIEW
  Total queries:        5412
  Successful execution: 5412 (100.0%)
  Failed execution:     0 (0.0%)

RESULT STATISTICS
  With results:    1010 (18.7%)
  Empty results:   4402 (81.3%)

RESULT COUNT DISTRIBUTION
      0 results:  4402 ################################
      1 results:   175 #
    2-5 results:   310 ##
   6-10 results:   106 
  11-50 results:    75 
    50+ results:   344 ##


In [5]:
# Close connection
driver.close()
print("Database connection closed")

# Show sample of the dataframe
df[["question", "answer", "result_count"]].head(10)

Database connection closed


,question,answer,result_count
0,Dược liệu Long nha thảo được sản xuất từ sinh ...,MATCH (o:ORGANISM) WHERE toLower(o.id) CONTAIN...,0
1,Dược liệu tác động lên cơ quan nào,MATCH (d:DRUG)-[:TREATS]->(dis:DISEASE) RETURN...,88
2,Dược liệu Long nha thảo có chỉ số kỹ thuật nào?,MATCH (d:DRUG)-[:HAS_STANDARD]->(s:STANDARD) W...,0
3,Dược liệu Long đởm chủ trị bệnh gì,MATCH (d:DRUG)-[:TREATS]->(dis:DISEASE) WHERE ...,0
4,Dược liệu Long đởm được sản xuất bằng phương p...,MATCH (d:DRUG)-[:PRODUCED_BY]->(p:PRODUCTION_M...,0
5,Dược liệu Long đởm được bảo quản ở điều kiện nào,MATCH (d:DRUG)-[:STORED_AT]->(s:STORAGE_CONDIT...,0
6,LONG ĐỞM được sản xuất từ vi sinh vật nào,MATCH (d:DRUG)-[:CONTAINS]->(o:ORGANISM) WHERE...,0
7,Kiên long đởm được sản xuất từ vi sinh vật nào,MATCH (d:DRUG)-[:PRODUCED_BY]->(o:ORGANISM) WH...,0
8,Phương pháp sắc ký đồ có chỉ số kỹ thuật nào?,MATCH (t:TEST_METHOD)-[:HAS_STANDARD]->(s:STAN...,0
9,Dược liệu Long đởm có chỉ số kỹ thuật nào,MATCH (d:DRUG)-[:HAS_STANDARD]->(s:STANDARD) W...,0


In [6]:
df.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')

In [7]:
valid_samples = df[df["result_count"] > 0].sample(5)

valid_samples.head()

,question,answer,result_count
525,Bột dược liệu có tiêu chuẩn nào?,MATCH (d:DRUG)-[:HAS_STANDARD]->(s:STANDARD) W...,1
4275,Phương pháp an toàn chung yêu cầu hóa chất nào,MATCH (t:TEST_METHOD)-[:REQUIRES]->(c:CHEMICAL...,1
2471,Tiêu chuẩn Chất chiết được trong dược liệu có ...,MATCH (s:STANDARD) WHERE toLower(s.id) CONTAIN...,1
733,Phương pháp Sắc ký yêu cầu hóa chất nào?,MATCH (t:TEST_METHOD)-[:REQUIRES]->(c:CHEMICAL...,251
2961,Phương pháp sắc ký lớp mỏng yêu cầu hóa chất nào,MATCH (t:TEST_METHOD)-[:REQUIRES]->(c:CHEMICAL...,251


## Valid sample Questions

- Phương pháp sắc ký lớp mỏng có tiêu chuẩn nào
- Phương pháp Sắc ký yêu cầu hóa chất nào?
- Dược liệu điều trị bệnh gì?
- Thuốc Đỗ trọng điều trị bệnh nào?